In [1]:
import pandas as pd
import numpy as np
import climb_jor
import climber_class
import grades_class

# Route recommender

In this notebook will perform the route recommending based on the user features

### Create the user from climber class

In [2]:
climber_1 = climber_class.climber()
climber_2 = climber_class.climber(grade = '7a+')
climber_2.set_attributes(cluster=[-3,4,11,-1,30,-29,3,2,8])
display(climber_2.get_data())

Climber class initialized
Climber class initialized


,climber_id,name,grade_fra,grade,grade_range,country,crag,sector,height
0,1,climber1,7a+,51,2,esp,montserrat,agulla del senglar,170


### Retrieve routes table

In [3]:
routes = pd.read_csv('../data/routes_rated.csv',low_memory=False, index_col=0)
routes.head(2)

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot
0,0,and,montserrat,prohibitivo,el mehir,-1,49.0,0,0.026886
1,1,and,montserrat,prohibitivo,el pas de la discordia,0,49.0,0,-0.032202


### Cleaning the table

Objective
- We will recommend 3 routes per each zone (country, crag, sector)

The logical tree will be
- Make 3 different filters for: sector, crag, country
- Filter tall recommended (if height<-1 and tall or if height>1 and small) tall is mean+5 (>180), short is mean-5 (<170)
- Filter by grade range, default +-2 (e.g. for a 7b grade => range[7a+,7b+]), if not routes ask to make the range bigger
- Order by overall rating
- Filter preferred clusters (If not then show other results)

#### a) make zone tables

In [4]:
climber_2.get_data()

,climber_id,name,grade_fra,grade,grade_range,country,crag,sector,height
0,1,climber1,7a+,51,2,esp,montserrat,agulla del senglar,170


In [5]:
routes_country = routes[routes.country == climber_2.get_data().country[0]]
routes_crag = routes[routes.crag == climber_2.get_data().crag[0]]
routes_sector = routes[routes.sector == climber_2.get_data().sector[0]]
print("Country routes: ", routes_country.shape[0])
print("Crag routes: ", routes_crag.shape[0])
print("Sector routes: ", routes_sector.shape[0])

Country routes:  15012
Crag routes:  812
Sector routes:  17


#### b) Filter tall recommend

In [6]:
climber_1.set_attributes(height=165)
climber_2.set_attributes(height=185)

Test with short

In [7]:
routes_country_tall = climb_jor.filter_tall(routes_df = routes_country, climber_usr = climber_1, show = True)
routes_crag_tall = climb_jor.filter_tall(routes_df = routes_crag, climber_usr = climber_1, show = True)
routes_sector_tall = climb_jor.filter_tall(routes_df = routes_sector, climber_usr = climber_1, show = True)

df routes:  12292
df routes:  661
df routes:  16


Test with tall

In [8]:
routes_country_tall = climb_jor.filter_tall(routes_df = routes_country, climber_usr = climber_2, show = True)
routes_crag_tall = climb_jor.filter_tall(routes_df = routes_crag, climber_usr = climber_2, show = True)
routes_sector_tall = climb_jor.filter_tall(routes_df = routes_sector, climber_usr = climber_2, show = True)

df routes:  12560
df routes:  712
df routes:  14


#### c) Filter grade range

In [9]:
routes_country_grade = climb_jor.filter_grade(routes_df = routes_country_tall, climber_usr = climber_2, show = True)
routes_crag_grade = climb_jor.filter_grade(routes_df = routes_crag_tall, climber_usr = climber_2, show = True)
routes_sector_grade = climb_jor.filter_grade(routes_df = routes_sector_tall, climber_usr = climber_2, show = True)

df routes:  1736
df routes:  122
df routes:  1


#### d) Order by overall rating

In [10]:
routes_country_rating = routes_country_grade.sort_values(by = 'rating_tot',ascending = False)
routes_crag_rating = routes_crag_grade.sort_values(by = 'rating_tot',ascending = False)
routes_sector_rating = routes_sector_grade.sort_values(by = 'rating_tot',ascending = False)

#### d) Filter clustering

In [11]:
routes_country_grade = climb_jor.filter_cluster(routes_df = routes_country_rating, climber_usr = climber_2, show = True)
routes_crag_grade = climb_jor.filter_cluster(routes_df = routes_crag_rating, climber_usr = climber_2, show = True)
routes_sector_grade = climb_jor.filter_cluster(routes_df = routes_sector_rating, climber_usr = climber_2, show = True)

df routes:  1736
df routes:  122
df routes:  1


In [12]:
climber_2.get_cluster_order()

[4, 2, 8, 1, 6, 7, 3, 0, 5]

#### f) 3 per zone

In [13]:
display(routes_country_grade.head(3))
display(routes_crag_grade.head(3))
display(routes_sector_grade.head(3))

,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
21911,21911,esp,rodellar,palomera,super remedios,51.000000,4,1.312488,0
16759,16759,esp,la pedriza,la peseta,el libro del fonambulista l,49.333333,4,1.312488,0
17018,17018,esp,lagos,cueva sala,la brecha,50.500000,4,1.036370,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19451,19451,esp,montserrat,el vermell,omaticaia,51.00,4,-0.020609,0
19632,19632,esp,montserrat,prohibitivo,el crosto,52.74,2,0.300076,0
19111,19111,esp,montserrat,clot del boixar,innus r,51.00,1,1.358676,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
18978,18978,esp,montserrat,agulla del senglar,el flautista,51.0,0,1.312488,0


## Create the function

In [14]:
def route_recommender(climber, show = False):
    '''
    This function returns the route recommendations for the user "climber"
    inputs:
    - climber: the user from the climber class
    - show: True for showing the log
    
    output:
    df with the recommendations       
    '''
    
    
    def recommender_filter(df, climber, show = True):
        routes = df.copy()
         # Climber Filters    
        routes_tall = climb_jor.filter_tall(routes_df = routes, climber_usr = climber, show = show)
        routes_grade = climb_jor.filter_grade(routes_df = routes_tall, climber_usr = climber, show = show)

        # Rating Order
        routes_rating = routes_grade.sort_values(by = 'rating_tot',ascending = False)

        # Clustering Order
        routes_grade = climb_jor.filter_cluster(routes_df = routes_rating, climber_usr = climber, show = show)

        return routes_grade
    
    
    routes = pd.read_csv('../data/routes_rated.csv',low_memory=False, index_col=0)
       
    # Region
    routes_country = routes[routes.country == climber.get_data().country.values[0]]
    routes_crag = routes[routes.crag == climber.get_data().crag.values[0]]
    routes_sector = routes[routes.sector == climber.get_data().sector.values[0]]
    
    # Recommen
    routes_country_rec = recommender_filter(routes_country, climber, show)
    routes_crag_rec = recommender_filter(routes_crag, climber, show)
    routes_sector_rec = recommender_filter(routes_sector, climber, show)
    
    display(routes_country_rec.head(3))
    display(routes_crag_rec.head(3))
    display(routes_sector_rec.head(3))
       

In [15]:
route_recommender(climber_2)

,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
21911,21911,esp,rodellar,palomera,super remedios,51.000000,4,1.312488,0
16759,16759,esp,la pedriza,la peseta,el libro del fonambulista l,49.333333,4,1.312488,0
17018,17018,esp,lagos,cueva sala,la brecha,50.500000,4,1.036370,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19451,19451,esp,montserrat,el vermell,omaticaia,51.00,4,-0.020609,0
19632,19632,esp,montserrat,prohibitivo,el crosto,52.74,2,0.300076,0
19111,19111,esp,montserrat,clot del boixar,innus r,51.00,1,1.358676,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
18978,18978,esp,montserrat,agulla del senglar,el flautista,51.0,0,1.312488,0


In [16]:
country = 'esp'
crag = 'montserrat'
sector = 'el vermell'

climber_3 = climber_class.climber(name = 'Jordi',
                                  grade = '7b+',
                                  grade_range = 4,
                                  cluster = [0,0,2,0,5,6,3,4,0],
                                  location = [country, crag, sector])

Climber class initialized


In [17]:
route_recommender(climber_3)

,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
18605,18605,esp,moclin,motherfucker,la cura de humildad,57.0,4,1.864724,0
21983,21983,esp,rosildos,la moreria,ivan el terrible,55.5,4,1.864724,0
24182,24182,esp,ubierna,entrada,amor de hermano,55.0,4,1.864724,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
18977,18977,esp,montserrat,agulla del senglar,discordia l,53.574074,2,0.482378,0
19632,19632,esp,montserrat,prohibitivo,el crosto,52.740000,2,0.300076,0
19407,19407,esp,montserrat,el vermell,la tele del menguele,52.666667,0,1.312488,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19407,19407,esp,montserrat,el vermell,la tele del menguele,52.666667,0,1.312488,0
19484,19484,esp,montserrat,el vermell,que remuga lavi,57.000000,0,1.036370,0
19471,19471,esp,montserrat,el vermell,polonesa,55.000000,0,0.760251,0


In [24]:
montse = climb_jor.Crag_search(routes, 'montserrat')

In [25]:
montse.cluster.value_counts()

0    424
1    291
3     88
4      7
2      2
Name: cluster, dtype: int64

In [34]:
montse[montse.tall_recommend_sum < -2]

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot
19029,19029,esp,montserrat,can jorba,guakamayo,-3,53.600000,0,0.225688
19219,19219,esp,montserrat,cova de larcada,good feeling,-3,49.928571,1,0.438320
19281,19281,esp,montserrat,el vermell,aniversari,-3,40.258065,1,0.199882
19508,19508,esp,montserrat,el vermell,shin shat,-3,50.807692,1,-0.148690
19552,19552,esp,montserrat,el vermell,via kpujo,-5,38.500000,3,0.025116
19646,19646,esp,montserrat,prohibitivo,kill bill,-4,50.555556,1,-0.127005


In [19]:
a,b,c = climber_2.route_recommender(show = True)

df routes:  12560
df routes:  1736
df routes:  1736
df routes:  712
df routes:  122
df routes:  122
df routes:  14
df routes:  1
df routes:  1


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
21911,21911,esp,rodellar,palomera,super remedios,51.000000,4,1.312488,0
16759,16759,esp,la pedriza,la peseta,el libro del fonambulista l,49.333333,4,1.312488,0
17018,17018,esp,lagos,cueva sala,la brecha,50.500000,4,1.036370,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19451,19451,esp,montserrat,el vermell,omaticaia,51.00,4,-0.020609,0
19632,19632,esp,montserrat,prohibitivo,el crosto,52.74,2,0.300076,0
19111,19111,esp,montserrat,clot del boixar,innus r,51.00,1,1.358676,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
18978,18978,esp,montserrat,agulla del senglar,el flautista,51.0,0,1.312488,0


In [20]:
cl = climber_class.climber()

a,b,c = cl.route_recommender(show = True)

Climber class initialized
df routes:  15012
df routes:  2047
df routes:  2047
df routes:  812
df routes:  127
df routes:  127
df routes:  17
df routes:  1
df routes:  1


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
22730,22730,esp,santibanez,matadero,viejas glorias,52.5,0,1.864724,0
12309,12309,esp,castellon,la cantera,trago amargo,53.5,0,1.864724,0
12244,12244,esp,capcanes,barranc del torto,estripats,53.5,0,1.864724,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19407,19407,esp,montserrat,el vermell,la tele del menguele,52.666667,0,1.312488,0
19617,19617,esp,montserrat,odio africano,directa africana,53.000000,0,0.760251,0
19471,19471,esp,montserrat,el vermell,polonesa,55.000000,0,0.760251,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
18977,18977,esp,montserrat,agulla del senglar,discordia l,53.574074,2,0.482378,0
